In [1]:
import sys
import pandas as pd
sys.path.append('..')
import os
import rbf_functions

In [2]:
rbfs = [rbf_functions.original_rbf,
        rbf_functions.squared_exponential_rbf,
        rbf_functions.inverse_quadratic_rbf,
        rbf_functions.inverse_multiquadric_rbf,
        rbf_functions.exponential_rbf,
        rbf_functions.matern32_rbf,
        rbf_functions.matern52_rbf,
       ]

# Load data

In [3]:
gddata = {}
eidata = {}
hvdata = {}
varlist = ['10', '20', '30', '40', '50', '60', '70', '80', '90', '100']
for entry in rbfs:
    rbf = entry.__name__
    df_gd = pd.read_csv(f"./gd/{rbf}_gd_all.csv")
    df_ei = pd.read_csv(f"./epind/{rbf}_ei_all.csv")
    df_hv = pd.read_csv(f"./hv/{rbf}_hv_global.csv")
    varlist = ['10', '20', '30', '40', '50', '60', '70', '80', '90', '100']
    gddata[rbf] = {}
    eidata[rbf] = {}
    hvdata[rbf] = {}
    for i in varlist: # add metric value at 100.000 nfe to dict for each seed
        gddata[rbf][i] = df_gd[i].dropna().iloc[-1] 
        eidata[rbf][i] = df_ei[i].dropna().iloc[-1]
        hvdata[rbf][i] = df_hv[i].dropna().iloc[-1]
df_gd_m = pd.DataFrame.from_dict(gddata, orient='index')
df_ei_m = pd.DataFrame.from_dict(eidata, orient='index')
df_hv_m = pd.DataFrame.from_dict(hvdata, orient='index')

In [8]:
global_hv = pd.read_csv('./hv/hypervolume_global_refset.csv', index_col='Unnamed: 0')
global_hv.iloc[0,0]

0.4148666686740452

In [10]:
df_hv_m /= global_hv.iloc[0,0]
df_hv_m

,10,20,30,40,50,60,70,80,90,100
original_rbf,0.632237,0.951296,0.960922,0.967530,0.963199,0.969625,0.962592,0.939670,0.960827,0.967008
squared_exponential_rbf,0.961888,0.959941,0.968866,0.969751,0.964346,0.964970,0.956514,0.968274,0.965126,0.960030
inverse_quadratic_rbf,0.952930,0.968920,0.966121,0.966836,0.969473,0.962436,0.966480,0.957823,0.963142,0.960125
inverse_multiquadric_rbf,0.964843,0.964953,0.964024,0.958134,0.951316,0.964225,0.960424,0.970197,0.599474,0.968026
exponential_rbf,0.956921,0.966841,0.602541,0.965282,0.965651,0.946534,0.963021,0.967293,0.952213,0.968545
matern32_rbf,0.950326,0.968734,0.952828,0.958184,0.969443,0.961540,0.964348,0.962150,0.970581,0.967355
matern52_rbf,0.965129,0.935532,0.965126,0.963372,0.963439,0.958827,0.962867,0.966154,0.956514,0.651577


In [11]:
import os

paretosets = {}
counts = {}
for entry in rbfs:
    name = entry.__name__
    # load variables
    varlist = []
    variables = []

    output_dir = os.path.abspath('./refsets/')
    
    for filename in os.listdir(output_dir):
        if filename == f'{name}_refset.csv':
            varlist.append(filename[:-4])
            df_vars = pd.read_csv(os.path.join(output_dir, filename))
            counts[name] = len(df_vars)
            paretosets[name] = df_vars
    variables = df_vars.values

counts

{'original_rbf': 32,
 'squared_exponential_rbf': 30,
 'inverse_quadratic_rbf': 31,
 'inverse_multiquadric_rbf': 30,
 'exponential_rbf': 31,
 'matern32_rbf': 30,
 'matern52_rbf': 32}

In [12]:
df_sol = pd.DataFrame.from_dict(counts, orient='index')
df_sol

,0
original_rbf,32
squared_exponential_rbf,30
inverse_quadratic_rbf,31
inverse_multiquadric_rbf,30
exponential_rbf,31
matern32_rbf,30
matern52_rbf,32


In [13]:
df_table = pd.DataFrame()
df_table["solutions"] = df_sol[0]
df_table["Generational Distance avg"] = df_gd_m.T.mean()
df_table["Epsilon Indicator avg"] = df_ei_m.T.mean()
df_table["Hypervolume avg"] = df_hv_m.T.mean()
df_table

,solutions,Generational Distance avg,Epsilon Indicator avg,Hypervolume avg
original_rbf,32,0.026647,0.351215,0.927491
squared_exponential_rbf,30,0.025668,0.406599,0.963971
inverse_quadratic_rbf,31,0.026272,0.437787,0.963429
inverse_multiquadric_rbf,30,0.024845,0.437820,0.926562
exponential_rbf,31,0.025584,0.362173,0.925484
matern32_rbf,30,0.024645,0.378529,0.962549
matern52_rbf,32,0.025515,0.360079,0.928854


In [14]:
import collections
df_global = pd.read_csv(f"./refsets/all_rbf_refset.csv")
global_refset = set([tuple(entry) for entry in df_global.values.tolist()])

contribution = collections.defaultdict(int)
for rbf in paretosets:
    for solution in paretosets[rbf].values.tolist():
        solution = tuple(solution)
        if solution in global_refset:
            contribution[rbf] +=1
    
contribution = {k:round(v/len(global_refset),3) for k,v in contribution.items()}

In [15]:
contribution

{'original_rbf': 0.172,
 'squared_exponential_rbf': 0.172,
 'inverse_quadratic_rbf': 0.103,
 'inverse_multiquadric_rbf': 0.241,
 'exponential_rbf': 0.138,
 'matern32_rbf': 0.069,
 'matern52_rbf': 0.103}

In [16]:
df_table['set contribution'] = pd.Series(contribution)

In [17]:
df_table

,solutions,Generational Distance avg,Epsilon Indicator avg,Hypervolume avg,set contribution
original_rbf,32,0.026647,0.351215,0.927491,0.172
squared_exponential_rbf,30,0.025668,0.406599,0.963971,0.172
inverse_quadratic_rbf,31,0.026272,0.437787,0.963429,0.103
inverse_multiquadric_rbf,30,0.024845,0.437820,0.926562,0.241
exponential_rbf,31,0.025584,0.362173,0.925484,0.138
matern32_rbf,30,0.024645,0.378529,0.962549,0.069
matern52_rbf,32,0.025515,0.360079,0.928854,0.103


In [18]:
df_table.to_latex()

/var/folders/_t/l6cr0r0d1m108xdrmrtz7stsb_csl0/T/ipykernel_70126/909359497.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_table.to_latex()


'\\begin{tabular}{lrrrrr}\n\\toprule\n{} &  solutions &  Generational Distance avg &  Epsilon Indicator avg &  Hypervolume avg &  set contribution \\\\\n\\midrule\noriginal\\_rbf             &         32 &                   0.026647 &               0.351215 &         0.927491 &             0.172 \\\\\nsquared\\_exponential\\_rbf  &         30 &                   0.025668 &               0.406599 &         0.963971 &             0.172 \\\\\ninverse\\_quadratic\\_rbf    &         31 &                   0.026272 &               0.437787 &         0.963429 &             0.103 \\\\\ninverse\\_multiquadric\\_rbf &         30 &                   0.024845 &               0.437820 &         0.926562 &             0.241 \\\\\nexponential\\_rbf          &         31 &                   0.025584 &               0.362173 &         0.925484 &             0.138 \\\\\nmatern32\\_rbf             &         30 &                   0.024645 &               0.378529 &         0.962549 &             0.069 \\